## What about Cart Size?

Another question to consider is, can we predict how large the next order will be? This can be answered in a similar way to the previous question about when the order will be - characterize each user according to their buying habits, but add in additional information regarding their most recent orders to inform the model regarding more recent trends.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

from sklearn import preprocessing
from sklearn import __version__ as sklearn_version
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

In [2]:
orders_all_filt = pd.read_csv('C:/Users/Max/Documents/Projects/Instacart Market Basket/Data/Data_Edits/orders_all_filt.csv')
df_prior = pd.read_csv('C:/Users/Max/Documents/Projects/Instacart Market Basket/Data/Data_Edits/df_prior.csv')

In [3]:
# Let's look at a specific order for reference
orders_all_filt[orders_all_filt.order_id==1187899]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
59,1187899,1,train,11,4,8,14.0,196,1,1
60,1187899,1,train,11,4,8,14.0,25133,2,1
61,1187899,1,train,11,4,8,14.0,38928,3,1
62,1187899,1,train,11,4,8,14.0,26405,4,1
63,1187899,1,train,11,4,8,14.0,39657,5,1
64,1187899,1,train,11,4,8,14.0,10258,6,1
65,1187899,1,train,11,4,8,14.0,13032,7,1
66,1187899,1,train,11,4,8,14.0,26088,8,1
67,1187899,1,train,11,4,8,14.0,27845,9,0
68,1187899,1,train,11,4,8,14.0,49235,10,1


In [4]:
print(orders_all_filt.user_id.nunique(), orders_all_filt.order_id.nunique())

print(df_prior.user_id.nunique(), df_prior.order_id.nunique())


131209 2178586
206209 3214874


Once again we need to cut out the most recent orders, as the cart size of the most recent orders are the ones we need to predict. Let's just cut down "orders_all_filt" because "df_prior" contains users we cut before.

In [5]:
orders_af = orders_all_filt[orders_all_filt.eval_set=='prior']

group2 = orders_af.groupby(['user_id','order_number']).count()

group2 #gets cart size per order per user, but nothing else

order_id  eval_set  order_dow  order_hour_of_day  \
user_id order_number                                                     
1       1                    5         5          5                  5   
        2                    6         6          6                  6   
        3                    5         5          5                  5   
        4                    5         5          5                  5   
        5                    8         8          8                  8   
...                        ...       ...        ...                ...   
206209  9                    3         3          3                  3   
        10                   9         9          9                  9   
        11                   8         8          8                  8   
        12                  20        20         20                 20   
        13                   9         9          9                  9   

                      days_since_prior_order  product_id  add_to_cart_order  \
user_id order_number                                                          
1       1                                  0           5                  5   
        2                                  6           6                  6   
        3                                  5           5                  5   
        4                                  5           5                  5   
        5                                  8           8                  8   
...                                      ...         ...                ...   
206209  9                                  3           3                  3   
        10                                 9           9                  9   
        11                                 8           8                  8   
        12                                20          20                 20   
        13                                 9           9                  9   

                      reordered  
user_id order_number             
1       1                     5  
        2                     6  
        3                     5  
        4                     5  
        5                     8  
...                         ...  
206209  9                     3  
        10                    9  
        11                    8  
        12                   20  
        13                    9  

[2047377 rows x 8 columns]

In [6]:
orders_af.order_number.max() 

99

In [7]:
orders_af.add_to_cart_order.max() #Finding the number of items in the largest order, the maximum order size

145

In [8]:
group2['cart_size'] = group2['reordered']
group2

order_id  eval_set  order_dow  order_hour_of_day  \
user_id order_number                                                     
1       1                    5         5          5                  5   
        2                    6         6          6                  6   
        3                    5         5          5                  5   
        4                    5         5          5                  5   
        5                    8         8          8                  8   
...                        ...       ...        ...                ...   
206209  9                    3         3          3                  3   
        10                   9         9          9                  9   
        11                   8         8          8                  8   
        12                  20        20         20                 20   
        13                   9         9          9                  9   

                      days_since_prior_order  product_id  add_to_cart_order  \
user_id order_number                                                          
1       1                                  0           5                  5   
        2                                  6           6                  6   
        3                                  5           5                  5   
        4                                  5           5                  5   
        5                                  8           8                  8   
...                                      ...         ...                ...   
206209  9                                  3           3                  3   
        10                                 9           9                  9   
        11                                 8           8                  8   
        12                                20          20                 20   
        13                                 9           9                  9   

                      reordered  cart_size  
user_id order_number                        
1       1                     5          5  
        2                     6          6  
        3                     5          5  
        4                     5          5  
        5                     8          8  
...                         ...        ...  
206209  9                     3          3  
        10                    9          9  
        11                    8          8  
        12                   20         20  
        13                    9          9  

[2047377 rows x 9 columns]

In [9]:
group2a = group2.groupby(['user_id','cart_size']).size()
group2a

user_id  cart_size
1        4            1
         5            4
         6            3
         8            1
         9            1
                     ..
206209   12           1
         13           1
         15           1
         16           1
         20           1
Length: 1021536, dtype: int64

In [10]:
group2_pcts = group2a.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

In [11]:
group2b = group2_pcts.unstack(level='cart_size')

group2b

cart_size,1,2,3,4,5,6,7,8,9,10,...,108,109,112,114,115,116,121,127,137,145
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,10.0,40.000000,30.000000,NaN,10.000000,10.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,7.142857,7.142857,NaN,NaN,14.285714,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,25.000000,NaN,NaN,NaN,25.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5.000000,NaN,5.000000,10.0,5.000000,NaN,5.000000,5.000000,10.000000,15.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.263158,15.789474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206200,4.347826,NaN,8.695652,NaN,NaN,4.347826,NaN,4.347826,8.695652,4.347826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Getting the most recent order per user
idx_1 = orders_all_filt.groupby(['user_id'])['order_number'].transform(max) == orders_all_filt['order_number']
groupie_1 = orders_all_filt[idx_1]

groupie_1

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
59,1187899,1,train,11,4,8,14.0,196,1,1
60,1187899,1,train,11,4,8,14.0,25133,2,1
61,1187899,1,train,11,4,8,14.0,38928,3,1
62,1187899,1,train,11,4,8,14.0,26405,4,1
63,1187899,1,train,11,4,8,14.0,39657,5,1
...,...,...,...,...,...,...,...,...,...,...
22026603,272231,206209,train,14,6,14,30.0,40603,4,0
22026604,272231,206209,train,14,6,14,30.0,15655,5,0
22026605,272231,206209,train,14,6,14,30.0,42606,6,0
22026606,272231,206209,train,14,6,14,30.0,37966,7,0


In [13]:
groupie_1b = groupie_1.groupby('user_id')['add_to_cart_order'].max()
groupie_1b

user_id
1         11
2         31
5          9
7          9
8         18
          ..
206199    22
206200    19
206203    13
206205    19
206209     8
Name: add_to_cart_order, Length: 131209, dtype: int64

In [14]:
group2c = group2b
group2c['prev'] = groupie_1b

group2c



cart_size,1,2,3,4,5,6,7,8,9,10,...,109,112,114,115,116,121,127,137,145,prev
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,10.0,40.000000,30.000000,NaN,10.000000,10.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
2,NaN,NaN,NaN,NaN,7.142857,7.142857,NaN,NaN,14.285714,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31
5,NaN,NaN,NaN,NaN,25.000000,NaN,NaN,NaN,25.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,5.000000,NaN,5.000000,10.0,5.000000,NaN,5.000000,5.000000,10.000000,15.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.263158,15.789474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
206200,4.347826,NaN,8.695652,NaN,NaN,4.347826,NaN,4.347826,8.695652,4.347826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
206203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13


In [15]:
# Repeating the above procedure to populate 3 additional columns of information
idx_2 = orders_all_filt.groupby(['user_id'])['order_number'].transform(max)-1 == orders_all_filt['order_number']
groupie_2 = orders_all_filt[idx_2]
groupie_2b = groupie_2.groupby('user_id')['add_to_cart_order'].max()
group2c['2_prev'] = groupie_2b

idx_3 = orders_all_filt.groupby(['user_id'])['order_number'].transform(max)-2 == orders_all_filt['order_number']
groupie_3 = orders_all_filt[idx_3]
groupie_3b = groupie_3.groupby('user_id')['add_to_cart_order'].max()
group2c['3_prev'] = groupie_3b

idx_4 = orders_all_filt.groupby(['user_id'])['order_number'].transform(max)-3 == orders_all_filt['order_number']
groupie_4 = orders_all_filt[idx_4]
groupie_4b = groupie_4.groupby('user_id')['add_to_cart_order'].max()
group2c['4_prev'] = groupie_4b



In [16]:
group2c['4_prev'].isna().sum()

group2c.fillna(0,inplace=True)

group2c



cart_size,1,2,3,4,5,6,7,8,9,10,...,115,116,121,127,137,145,prev,2_prev,3_prev,4_prev
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.000000,10.0,40.000000,30.000000,0.000000,10.000000,10.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,11,9,6,6
2,0.000000,0.000000,0.000000,0.0,7.142857,7.142857,0.000000,0.000000,14.285714,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,31,16,9,19
5,0.000000,0.000000,0.000000,0.0,25.000000,0.000000,0.000000,0.000000,25.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,9,12,5,9
7,5.000000,0.000000,5.000000,10.0,5.000000,0.000000,5.000000,5.000000,10.000000,15.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,9,12,5,13
8,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,18,13,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206199,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,5.263158,15.789474,...,0.0,0.0,0.0,0.0,0.0,0.0,22,28,23,25
206200,4.347826,0.000000,8.695652,0.0,0.000000,4.347826,0.000000,4.347826,8.695652,4.347826,...,0.0,0.0,0.0,0.0,0.0,0.0,19,30,27,9
206203,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,13,20,29,21


In [17]:
group2d = pd.get_dummies(data=group2c, columns = ['2_prev','3_prev','4_prev'])

group2d

,1,2,3,4,5,6,7,8,9,10,...,4_prev_73,4_prev_76,4_prev_77,4_prev_79,4_prev_81,4_prev_84,4_prev_86,4_prev_89,4_prev_91,4_prev_102
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.000000,10.0,40.000000,30.000000,0.000000,10.000000,10.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0.000000,0.000000,0.0,7.142857,7.142857,0.000000,0.000000,14.285714,0.000000,...,0,0,0,0,0,0,0,0,0,0
5,0.000000,0.000000,0.000000,0.0,25.000000,0.000000,0.000000,0.000000,25.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
7,5.000000,0.000000,5.000000,10.0,5.000000,0.000000,5.000000,5.000000,10.000000,15.000000,...,0,0,0,0,0,0,0,0,0,0
8,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206199,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,5.263158,15.789474,...,0,0,0,0,0,0,0,0,0,0
206200,4.347826,0.000000,8.695652,0.0,0.000000,4.347826,0.000000,4.347826,8.695652,4.347826,...,0,0,0,0,0,0,0,0,0,0
206203,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0


Now we have a dataframe that categorizes all users by their cart sizes in a way that isn't dependent on how many carts they have ordered but also captures information about their most recent order sizes.

In [18]:
X2 = group2c.drop(columns='prev')
y2 = group2c.prev

X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2,
                                                    test_size=0.3, 
                                                    random_state=1)

In [19]:
lm2 = LinearRegression().fit(X2_train, y2_train)



In [21]:
y2_train_pred = lm2.predict(X2_train)
y2_test_pred = lm2.predict(X2_test)

mean_squared_error(y2_train, y2_train_pred), mean_squared_error(y2_test, y2_test_pred)

(31.89691781304069, 7.122037896861718e+17)

In [22]:
rfr2 = RandomForestRegressor()
rfr2.fit(X2_train,y2_train)


RandomForestRegressor()

In [23]:

y2_pred = rfr2.predict(X2_test)
y2_train_pred = rfr2.predict(X2_train)

In [24]:
mean_squared_error(y2_train, y2_train_pred), mean_squared_error(y2_test,y2_pred)



(5.81537814645412, 34.340267076498264)

Somehow the linear regression model more or less overfit on the training set and essentially failed to make an accurate prediction on the test data (MSE = 7.1e17), but using random forest regression could predict cart size within approximately 6 items. Should this be a classification task instead? We're looking at discrete cart sizes, so each different number of items in a given cart could be considered as a category (so we'd be predicting which "category" a person's cart would fall into) as opposed to a continuous variable.



In [25]:
rfc = RandomForestClassifier()
rfc.fit(X2_train,y2_train)

RandomForestClassifier()

In [26]:
y2_class_pred = rfc.predict(X2_test)

print("Accuracy:", metrics.accuracy_score(y2_test,y2_class_pred))



Accuracy: 0.10232959886187537


What about comparing these results with those using one-hot encoding?

In [27]:
X2d = group2d.drop(columns='prev')
y2d = group2d.prev

X2d_train, X2d_test, y2d_train, y2d_test = train_test_split(X2d,y2d,
                                                    test_size=0.3, 
                                                    random_state=1)

In [28]:
rfr2d = RandomForestRegressor()
rfr2d.fit(X2d_train,y2d_train)

RandomForestRegressor()

In [29]:
y2d_pred = rfr2d.predict(X2d_test)
mean_squared_error(y2d_test,y2d_pred)

35.53954008218967

In [ ]:
# Too big: 7.34MB memory
#rfc2d = RandomForestClassifier()
#rfc2d.fit(X2d_train,y2d_train)

#y2d_class_pred = rfc2d.predict(X2d_test)
#print("Accuracy:", metrics.accuracy_score(y2_test,y2_class_pred))